In [80]:
import pandas as pd
navigator = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1)'
url_base = 'https://sofifa.com/player/'
from bs4 import BeautifulSoup
import requests

import pandas as pd
import streamlit as st
import streamlit_authenticator as stauth
from streamlit_option_menu import option_menu
import ast
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
from bs4 import BeautifulSoup
import pickle
import requests
import re
from bs4 import BeautifulSoup
import requests
import plotly.express as px

In [87]:
df = pd.read_csv('/Users/kilian/Documents/GitHub/Projet-3/STREAMLIT/BD/players_3120.csv')

In [88]:
df = df.sample(1)
df

,name,Age,Overall rating,Potential,Team & Contract,ID,Height,Weight,foot,Best overall,...,GK Reflexes,Total stats,Base stats,International reputation,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defending / Pace,Unnamed: 64
997,L. Barreiro Martins CM CDM,23,76,82,FSV Mainz 05\n2018 ~ 2024,245253,"174cm / 5'9""",68kg / 150lbs,Right,78,...,6,1955,416,2,68,61,69,80,73,NaN


In [89]:
def poids(x):
    texte = x
    chiffre = re.findall("\d+", texte)
    resultat = chiffre[0]
    return resultat

def calcul(value):

    if value == []:
        value = value
    else:
        if type(value) == list:
            value = ''.join(value)
        
        if ' ' in value:
            value = value.replace(' ', '')
            
        if '+' in value:
            a, b = value.split('+')
            return int(a) + int(b)
        elif '-' in value:
            a, b = value.split('-')
            return int(a) - int(b)
        
        value = int(value)
    
    return value

<>:3: SyntaxWarning:

invalid escape sequence '\d'

<>:3: SyntaxWarning:

invalid escape sequence '\d'

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_19440/1552836384.py:3: SyntaxWarning:

invalid escape sequence '\d'



In [90]:
def capacite(id):
    
    navigator = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1)'
    url_base = 'https://sofifa.com/player/'

    id = int(id)
    url_finale_title = f'{url_base}{id}'
    html_title = requests.get(url_finale_title, headers={'User-Agent': navigator})
    html_title2 = html_title.content
    soup_title = BeautifulSoup(html_title2, 'html.parser')

    dico = {}

    # FIN DE CONTRAT

    for balise_parent in soup_title.find_all('div', class_='grid attribute'):
        if 'Contract valid until' in balise_parent.get_text().strip():
                end_contract = balise_parent.get_text().strip()

    try:
        end_year = max(re.findall('\d{4,}', end_contract))
        end_year = int(end_year)
    except:
        end_year = 'Unknown'
    
    dico.update({'Team & Contract' : end_year})

    # SALAIRE ET COUT TRANSFERT

    for balise_parent2 in soup_title.find_all('div', class_='col'):
        if 'Value' in balise_parent2.get_text().strip():
            valeur = balise_parent2.get_text().strip()
        if 'Wage' in balise_parent2.get_text().strip():
            salary = balise_parent2.get_text().strip()

    valeur = valeur[:-5]
    salary = salary[:-4]
                
    chiffres = [valeur, salary]
    new_chiffres = []

    for element in chiffres:
        if 'M' not in element and 'K' not in element:
            new_chiffres.append(int(element.replace('€', '')))
        elif 'K' in element and '.' not in element:
            new_chiffres.append(int(element.replace('K', '000').replace('€', '')))
        elif 'K' in element and '.'  in element:
            new_chiffres.append(int(element.replace('K', '00').replace('€', '')))
        elif 'M' in element and '.' not in element:
            new_chiffres.append(int(element.replace('M', '000000').replace('€', '')))
        elif 'M' in element and '.' in element:
            new_chiffres.append(int(element.replace('M', '00000').replace('.', '').replace('€', '')))

    valeur = int(new_chiffres[0])

    salary = int(new_chiffres[1])

    dico.update({'Value': valeur,
                 'Wage': salary})
    
    # NOM
    
    name = ''

    for balise_parent in soup_title.find_all('div', class_='profile clearfix'):
        for balise_parent2 in soup_title.find_all('h1'):
            name += balise_parent2.get_text().strip() + ', '

    name = name[:-2]

    dico.update({'name' : name})

    # POTENTIAL


    texte = ""
    for balise_parent2 in soup_title.find_all('div', class_= 'grid'):
        if 'Potential' in balise_parent2.get_text().strip():
            texte += balise_parent2.get_text().strip()

    colonnes = ['Potential']
    for element in colonnes:
        debut = "(\d{2,2} )"
        chiffre = re.findall(f"{debut}{element}", texte)
        chiffre = "".join(chiffre)
        dico.update({element : chiffre})

    for element in colonnes:
        if dico[element] == '':
            debut = r'(\d\d [+-]\d)'
            chiffre = re.findall(f"{debut}{element}", texte)
            dico.update({element : calcul(chiffre)})

    for element in colonnes:
        if dico[element] == []:
            debut = "(\d{1,1} )"
            chiffre = re.findall(f"{debut}{element}", texte)
            dico.update({element : chiffre})

    for element in colonnes:
        if dico[element] == []:
            dico[element] = 0
        elif type(dico[element]) == list:
            dico[element] = int(''.join(dico[element]))

    # OVERALL


    texte = ""
    for balise_parent2 in soup_title.find_all('div', class_= 'grid'):
        if 'Overall rating' in balise_parent2.get_text().strip():
            texte += balise_parent2.get_text().strip()
    
    colonnes = ['Overall rating']

    for element in colonnes:
        debut = "(\d{2,2} )"
        chiffre = re.findall(f"{debut}{element}", texte)
        chiffre = "".join(chiffre)
        dico.update({element : chiffre})

    for element in colonnes:
        if dico[element] == '':
            debut = r'(\d\d [+-]\d)'
            chiffre = re.findall(f"{debut}{element}", texte)
            dico.update({element : calcul(chiffre)})

    for element in colonnes:
        if dico[element] == []:
            debut = "(\d{1,1} )"
            chiffre = re.findall(f"{debut}{element}", texte)
            dico.update({element : chiffre})

    for element in colonnes:
        if dico[element] == []:
            dico[element] = 0
        # elif ' ' in dico[element] :
        #     dico[element] = dico[element].replace(' ', '')
        if type(dico[element]) == list:
            dico[element] = int(''.join(dico[element]))


    # CAPACITE
    
    colonnes = list(df.columns)[16:-9]

    colonnes.remove('Total skill')
    colonnes.remove('Total movement')
    colonnes.remove('Total power')
    colonnes.remove('Total mentality')
    colonnes.remove('Total defending')
    colonnes.remove('Total goalkeeping')

    url_finale_title = f'{url_base}{id}'
    html_title = requests.get(url_finale_title, headers={'User-Agent': navigator})
    html_title2 = html_title.content
    soup_title = BeautifulSoup(html_title2, 'html.parser')

    texte = ''
    for balise_parent in soup_title.find_all('span'):
        texte += (balise_parent.get_text().strip())
    

    for element in colonnes:
        debut = "(\d{2,2})"
        chiffre = re.findall(f"{debut}{element}", texte)
        dico.update({element : chiffre})
        
        if dico[element] == [] or dico[element] == '':
            debut = r'(\d\d[+-]\d)'
            chiffre = re.findall(f"{debut}{element}", texte)
            dico.update({element : calcul(chiffre)})
        
        if dico[element] == [] or dico[element] == '':
            debut = "(\d{1,1})"
            chiffre = re.findall(f"{debut}{element}", texte)
            dico.update({element : chiffre})


    for element in colonnes:
        if type(dico[element]) == list:
            dico[element] = ''.join(dico[element])
        if dico[element] != '' or len(dico[element]) <= 3:
            dico[element] = int(dico[element])

    return dico


<>:21: SyntaxWarning:

invalid escape sequence '\d'

<>:83: SyntaxWarning:

invalid escape sequence '\d'

<>:96: SyntaxWarning:

invalid escape sequence '\d'

<>:117: SyntaxWarning:

invalid escape sequence '\d'

<>:130: SyntaxWarning:

invalid escape sequence '\d'

<>:165: SyntaxWarning:

invalid escape sequence '\d'

<>:175: SyntaxWarning:

invalid escape sequence '\d'

<>:21: SyntaxWarning:

invalid escape sequence '\d'

<>:83: SyntaxWarning:

invalid escape sequence '\d'

<>:96: SyntaxWarning:

invalid escape sequence '\d'

<>:117: SyntaxWarning:

invalid escape sequence '\d'

<>:130: SyntaxWarning:

invalid escape sequence '\d'

<>:165: SyntaxWarning:

invalid escape sequence '\d'

<>:175: SyntaxWarning:

invalid escape sequence '\d'

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_19440/2577867916.py:21: SyntaxWarning:

invalid escape sequence '\d'

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_19440/2577867916.py:83: SyntaxWarning:

invalid escape sequenc

In [91]:
for n in range(len(df['ID'])):
    dico = capacite(df['ID'].iloc[n])
    for element in df.columns:
        if element in dico.keys():
            df[element].iloc[n] = dico[element]

df['Best position'] = df['Best position'].apply(lambda x : x.replace('RWB', 'RB').replace('LWB', 'LB')).replace('ST', 'CF')
df['Height'] = df['Height'].apply(lambda x : int(x[:3]))
df['Pied droit'] = df['foot'].apply(lambda x : 1 if x == 'Right' else 0)
df['Weight'] = df['Weight'].apply(poids)
df = df.drop(['foot', 'Unnamed: 64'], axis = 1)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_19440/4171990797.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_19440/4171990797.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_19440/4171990797.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipyker

In [76]:
# POUR STREAMLIT

colonnes = list(df.columns)[16:-9]

attacking = colonnes[0:5]
skill = colonnes[5:10]
movement = colonnes[10:15]
power = colonnes[15:20]
mentality = colonnes[20:26]
defending = colonnes[26:29]
goalkeeping = colonnes[29:34]
attacking, skill, movement, power, mentality, defending, goalkeeping

dico_colonnes = {'Attacking' : float(df[attacking].iloc[0].mean()),
        'Skill' : float(df[skill].iloc[0].mean()),
        'Movement' : float(df[movement].iloc[0].mean()),
        'Power' : float(df[power].iloc[0].mean()),
        'Mentality' : float(df[mentality].iloc[0].mean()),
        'Defending': float(df[defending].iloc[0].mean()),
        'Goalkeeping' : float(df[goalkeeping].iloc[0].mean())}


data = pd.DataFrame(df[attacking].iloc[3]).reset_index()

colonnes_data = list(data.columns)

data = data.rename({colonnes_data[0] : 'theta',
                    colonnes_data[1] : 'r'}, axis = 1)

list(data['theta']), list(data['r'])

fig = px.line_polar(df.iloc[n], r = dico_colonnes.values(), theta= dico_colonnes.keys(), line_close=True)

fig.show()